In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import numpy as np
import re
from tqdm import tqdm  # Pour la barre de progression

class EmotionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.labels = dataframe.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def parse_labels(self, label_str):
        numbers = re.findall(r'\d+', label_str)
        return [int(num) for num in numbers]

    def __getitem__(self, index):
        text = str(self.text[index])
        label_str = str(self.labels[index])
        labels = self.parse_labels(label_str)

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        label_vector = torch.zeros(28)
        for label in labels:
            label_vector[label] = 1

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label_vector
        }

def train_model(train_subset_size=10000):  # Paramètre pour limiter la taille des données
    # Charger et échantillonner les données
    train_df = pd.read_csv('go_emotions_train.csv')
    if train_subset_size:
        train_df = train_df.sample(n=min(train_subset_size, len(train_df)), random_state=42)
        train_df = train_df.reset_index(drop=True)

    val_df = pd.read_csv('go_emotions_validation.csv')

    # Configuration du device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Utilisation de : {device}")

    # Initialiser le tokenizer et le modèle
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=28,
        problem_type="multi_label_classification"
    )

    # Créer les datasets
    train_dataset = EmotionDataset(train_df, tokenizer)
    val_dataset = EmotionDataset(val_df, tokenizer)

    # Augmenter la taille du batch et utiliser plus de workers pour le chargement des données
    train_loader = DataLoader(
        train_dataset,
        batch_size=32,  # Augmenté de 16 à 32
        shuffle=True,
        num_workers=4,  # Utilise plusieurs processus pour charger les données
        pin_memory=True if torch.cuda.is_available() else False  # Optimisation pour GPU
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=32,
        num_workers=4,
        pin_memory=True if torch.cuda.is_available() else False
    )

    # Configurer l'optimiseur avec un learning rate plus élevé
    optimizer = AdamW(model.parameters(), lr=5e-5)  # Learning rate augmenté

    # Déplacer le modèle sur GPU/CPU
    model.to(device)

    # Entraînement
    num_epochs = 5 # Réduit de 3 à 2 époques

    for epoch in range(num_epochs):
        model.train()
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}')

        for batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels.float()
            )

            loss = outputs.loss
            loss.backward()
            optimizer.step()

            # Mettre à jour la barre de progression avec la loss
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    # Sauvegarder le modèle
    model.save_pretrained('model/')
    tokenizer.save_pretrained('model/')

train_model()

Utilisation de : cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch imp